# Inequality on Health Impacts from Air Pollution in Canadian Cities
## 323 Final Project

In [119]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

from shapely.geometry import Point, shape, Polygon, MultiPolygon
import fiona

%matplotlib inline

In [41]:
gini_import = pd.read_excel("./Data/gini.xlsx")

In [42]:
homicide_import = pd.read_excel("./Data/Homicide rate.xlsx", skipfooter = 4)

In [43]:
air_quality_import = pd.read_csv("./Data/WHO_air_quality_database_2018_v14_merged.csv")

In [86]:
health_names = ["Hamilton", "Kelowna", "Montreal", "Ottawa", "Quebec", "Saguenay", "Sherbrooke", "Toronto", "Trois-Rivieres", "Vancouver", "Waterloo", "Windsor", "Winnipeg"]
health_data_array = {}
selected_rows = ["Perceived health; very good or excellent (%)", "Lung cancer incidence (per 100000 population)", "Current smoker; daily or occasional (%)", "Current smoker; daily (%)", "Leisure-time physical activity; moderately active or active (%)", "Second-hand smoke; exposure at home (%)", "Second-hand smoke; exposure in vehicles and/or public places (%)", "Asthma (%)", "Lung cancer; deaths (per 100000 population)", "  Respiratory diseases; deaths (per 100000 population)", "Low income (%)"]
for name in health_names:
    health_data_array[name] = pd.read_csv("./Data/" + name + "2013.csv", header = 1, skipfooter = 13, engine='python', encoding = "latin-1").replace(np.NaN, 0).replace("..", 0)
    health_data_array[name]["Characteristics"] = health_data_array[name]["Characteristics"].str.strip()
    health_data_array[name] = health_data_array[name].loc[health_data_array[name]["Characteristics"].isin(selected_rows)]


,Topic,Characteristics,Total,Flag_Total,Male,Flag_Male,Female,Flag_Female,Total.1,Flag_Total.1,Male.1,Flag_Male.1,Female.1,Flag_Female.1
0,Well-being,Perceived health; very good or excellent (%),59.7,0,60.5,0,59.0,0,56.8,0,58.0,0,55.7,0
8,Health Conditions,Asthma (%),8.4,0,5.8,E,10.8,E,9.7,0,8.6,E,10.8,E
22,Health Conditions,Lung cancer incidence (per 100000 population),55.5,0,68.5,0,45.8,0,0.0,0,0.0,0,0.0,0
25,Health Behaviours,Current smoker; daily or occasional (%),20.5,0,24.5,0,16.9,0,19.6,0,19.3,0,20.0,0
26,Health Behaviours,Current smoker; daily (%),16.9,0,19.8,0,14.3,0,15.9,0,16.1,0,15.8,0
28,Health Behaviours,Leisure-time physical activity; moderately act...,57.5,0,61.5,0,53.7,0,52.4,0,54.0,0,50.9,0
53,Environmental Factors,Second-hand smoke; exposure at home (%),4.8,0,6.1,E,3.6,E,5.2,0,6.9,E,3.7,E
54,Environmental Factors,Second-hand smoke; exposure in vehicles and/or...,15.6,0,17.3,0,14.2,0,16.4,0,20.4,0,12.7,E
61,Deaths,Lung cancer; deaths (per 100000 population),42.7,0,53.9,0,34.6,0,0.0,0,0.0,0,0.0,0
83,Living and Working Conditions,Low income (%),15.7,0,14.6,0,16.7,0,12.0,0,10.4,0,13.5,0


In [87]:
air_quality_canada = air_quality_import.loc[air_quality_import["Country"] == "Canada"]
# air_quality = air_quality_canada.loc
gini_import["Census Metropolitan Area "] = gini_import["Census Metropolitan Area "].str.strip()
cities = gini_import["Census Metropolitan Area "]
air_quality_cities = air_quality_canada.loc[air_quality_canada["City"].isin(cities)]
air_quality_cities = air_quality_cities.append(air_quality_canada.loc[air_quality_canada["City"] == "Ottawa"])
air_quality_cities = air_quality_cities.append(air_quality_canada.loc[air_quality_canada["City"] == "Abbotsford"])
air_quality_cities = air_quality_cities.append(air_quality_canada.loc[air_quality_canada["City"] == "Kitchener"])
air_quality_cities = air_quality_cities.append(air_quality_canada.loc[air_quality_canada["City"] == "Montreal"])
air_quality_cities = air_quality_cities.append(air_quality_canada.loc[air_quality_canada["City"] == "Quebec"])
air_quality_cities = air_quality_cities.append(air_quality_canada.loc[air_quality_canada["City"] == "Trois-Rivieres"])

In [88]:
cities_df = gpd.read_file("https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/files-fichiers/2016/lcma000b16a_e.zip")
canada_land_df = gpd.read_file("https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/files-fichiers/2016/lpr_000b16a_e.zip")
canada_water_df = gpd.read_file("https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/files-fichiers/2016/lpr_000a16a_e.zip")

D:\Program Data\Anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [89]:
cities_df["CMANAME"] = cities_df["CMANAME"].apply(lambda name: name if isinstance(name, str) else name.decode("latin-1"))

In [203]:
# fig, gax = plt.subplots(figsize = (15,8))
# canada_water_df.plot(ax = gax, edgecolor = 'black', color = 'lightblue')
# canada_land_df.plot(ax = gax, edgecolor = 'black', color = 'greenyellow')
city_points = cities_df.copy()
for name in air_quality_cities["City"]:
    city_row = cities_df.loc[cities_df["CMANAME"] == name]
    if len(city_row) == 0:
        continue
    city_index = list(city_row.index)[0]
    city_shape = cities_df.loc[city_index]["geometry"]
    poly = ""
    if isinstance(city_shape, Polygon):
        poly = city_shape
    else:
        poly = list(city_shape)[0]
    x, y = poly.exterior.coords.xy
    city_points.at[city_index, "geometry"] = Point(x[0], y[0])

city_points
    
# plt.plot()

<ipython-input-203-c3cb4fe48bcd>:15: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  poly = list(city_shape)[0]
<ipython-input-203-c3cb4fe48bcd>:15: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  poly = list(city_shape)[0]


,CMAUID,CMAPUID,CMANAME,CMATYPE,PRUID,PRNAME,geometry
0,512,35512,Brockville,D,35,Ontario,"MULTIPOLYGON (((7494625.237 1128257.371, 74950..."
1,442,24442,Trois-Rivières,B,24,b'Quebec / Qu\xe9bec',"MULTIPOLYGON (((7685035.983 1380936.549, 76866..."
2,444,24444,Shawinigan,D,24,b'Quebec / Qu\xe9bec',"POLYGON ((7634181.689 1409252.829, 7634554.614..."
3,447,24447,Drummondville,K,24,b'Quebec / Qu\xe9bec',"POLYGON ((7705178.249 1332485.077, 7705186.949..."
4,450,24450,Granby,K,24,b'Quebec / Qu\xe9bec',"POLYGON ((7703435.440 1270659.020, 7703893.906..."
...,...,...,...,...,...,...,...
151,845,48845,Cold Lake,D,48,Alberta,"POLYGON ((5048702.580 2200109.577, 5048763.483..."
152,850,48850,Grande Prairie,K,48,Alberta,"POLYGON ((4566556.417 2468181.671, 4565974.171..."
153,860,48860,Wood Buffalo,K,48,Alberta,"POLYGON ((5154537.534 2795507.449, 5155183.874..."
154,865,48865,Wetaskiwin,D,48,Alberta,"POLYGON ((4804472.494 2108080.594, 4804278.609..."


In [197]:
# poly = ""
# test = cities_df["geometry"][0]
# if isinstance(test, Polygon):
#     poly = test
# else:
#     poly = list(test)[0]
# x, y = poly.exterior.coords.xy
# x[0]
# y[0]
# city = cities_df.loc[cities_df["CMANAME"] == name]["geometry"]
# city[list(city.index)[0]]
# (49.2827, 123.1207).apply(Point)
Point(49.2827, 123.1207)
city_row = cities_df.loc[cities_df["CMANAME"] == "Vancouver"]
display(city_row)
city_index = city_row.index
display(list(city_index)[0])
city_shape = cities_df.loc[city_index]["geometry"]
display(list(city_shape.index)[0])

,CMAUID,CMAPUID,CMANAME,CMATYPE,PRUID,PRNAME,geometry
129,933,59933,Vancouver,B,59,British Columbia / Colombie-Britannique,"MULTIPOLYGON (((4042996.157 2027876.066, 40452..."


129

[129]